In this notebook I am going to discuss about what is time series and some basic concepts of time series analysis. This notebook will guide you through some concepts of time series analysis and how you can forecast by applying time series models. I am using gold price dataset to predict future prices. The prerequisite is here that you are already familiar with basic statistics.
                                                                     

**Time Series Analysis :**
                        Time series analysis is a statistical technique that deals with time series data, or trend analysis. Time series data means that data is in a series of particular time periods or intervals.
                        


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

About the dataset :
> The dataset consist many attributes but I have only used only gold prices for this notebook.

In [ ]:
data = pd.read_csv('../input/gold-price-data/gld_price_data.csv')
gold_data = data[['Date','GLD']]
gold_data.columns = ['date','gold_price']
gold_data.head()

The first step is to set date column as the index in the dataframe. For this, first we convert dates in pandas datatime using pd.to_datetime function and then set it as the index of dataframe.

In [ ]:
gold_data.index = pd.to_datetime(gold_data['date'])
gold_data.drop('date',axis=1,inplace = True)

We can select data based on year, month and date by setting the date as index. It is very useful if you set date as index in pandas Dataframe. Here I am selecting January,2018 to see what are the prices of gold in that particular month.

In [ ]:
gold_data['2018-01']

Now we plot a chart to see how the prices of gold varies from 2008 to 2018. Plotting a chart of series help us to visualise data take a quick glance to get an intution of dataset. Here we can clearly see that gold prices are very high in the time period of 2011 to 2013. For plotting a graph we use of python matplotlib library.

In [ ]:
import matplotlib.pyplot as plt
plt.plot(gold_data)
plt.xlabel('Year')
plt.ylabel('Price [USD]')
plt.title('Gold Prices')
plt.grid()
plt.show()

### AutoCorrelation : 

<b> Autocorrelation is the correlation of a point in the series with a point with lag taken as one day of the same series. </b> 
If a series show a positive autocorrelation then we series is momentum or trend following and if a series show negative correlation then we say series is mean reversing.

In python we can use pandas autocorr() function to calculate autocorrelation of a series.

In [ ]:
print(gold_data['gold_price'].autocorr())

Here the autocorrelation is positive so we can conclude that the series here is a trend following series.

### Autocorrelation Function : 
The sample autocorrelation function function show entire correlation function for different lags. So the autocorrelation function is a funtion of lag.
By using statsmodels python library we visualise aucorrelation for different lags. Autocorelation fuction help us to choose how many lags we can use for forcasting. 

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(gold_data['gold_price'],lags=20,alpha=0.5)
plt.show()

We can see that we can use lag of 20 days to forcast gold values because they have strongly positive autocorrelated.

### White Noise :
A noise time series simply a sequence of uncorrelated random variable that was identically distributed and follows a normal distribution with mean as zero.. Stock market are often modeled as white noise.

- We can't forecast future observations for white noise based on past values because autocorrelations at all lags are 0.

### Random Walk :

A series is said to be a random walk if a point of a series does not depend to previous point or entry. Every next point is random and we can not correlate them.
Example : Today's price = Yesterday's price + Noise <br>
  &emsp; &emsp; &emsp; &emsp;             P<sub>t</sub>  &emsp; &emsp;= &emsp; &emsp; P<sub>t-1</sub> &emsp; +&emsp; e<sub>t</sub>

Change in price in is white noise : <br>
   &emsp; &emsp; &emsp; &emsp;             P<sub>t</sub>  &emsp; &emsp;- &emsp;  P<sub>t-1</sub> &emsp; = &emsp;&emsp;e<sub>t</sub>
   
  - We can not forcast a random walk
  - Best forcast for a Random walk is today price or current point

### Augmented Dickey Fuller test :
This test is usefull to check weather a series is random walk or not. The null hypothesis is that the series follow a random walk. Therefore, a low p-value (say less then 5%) means we can reject null hyposthesis that time series is a random walk.
<br>
Test :  &emsp; &emsp; &emsp; &emsp;             P<sub>t</sub>  &emsp; - &emsp;  P<sub>t-1</sub> &emsp; = $\alpha$  &emsp;+  &emsp;$\beta$  P<sub>t-1</sub>  &emsp; + &emsp;e<sub>t</sub>

The eqution is only for one lag which for simple Dickey Fuller Test. We can add more lags to make the test augmented dickey fuller test. 


In [ ]:
from statsmodels.tsa.stattools import adfuller
results = adfuller(gold_data['gold_price'])
print('The p-value of the test on prices is: ' + str(results[1]))


According to this test, we cannot reject the hypothesis that Gold prices follow a random walk.

There is function a avialable in pandas pct_change() that calculates the percentage change between the current and a prior element. This function by default calculates the percentage change from the immediately previous row. It output first row as NaN so we have to apply dropna().
Here we test percent change in gold price to check that series is a random walk or not.

In [ ]:
from statsmodels.tsa.stattools import adfuller
results = adfuller(gold_data.pct_change().dropna())
print('The p-value of the test on prices is: ' + str(results[1]))


The p-value we get is less then 5% so we reject this null hypothesis. Percent change in gold prices not follow random walk

### Stationarity :
- <b>Strong Stationary :</b> A series is strong sationary if the entire distribution is time invariant. Joint distribtuion of the observations does not depend on time.
- <b>Weak Stationary</b> : A stationary process has the property that the mean, variance and autocorrelation structure do not change over time. 
    For autocorrelation, corr( X<sub>t</sub>, X<sub>t - $\tau$</sub>)is only function of $\tau$ and $\tau$ is a notation for lag.
    <br>
    
If a process non stationary then it become difficult to model.
- If parameters vary with time then  it results in too many parameters that needs to be estimate.
- We can only estimate model with a few parameters.

Random walk is a common type of non-stationary. <br>
Seasonal data is also non-stationary beacause mean varies with time.
    
<b>Therefore few technique to make a series stationary.</b> 
- We can take difference (df.diff() in python) then new series is a stationary series.
- If we take only log of the series then it remove only growth of the time series and if we take first log and then difference the it become more stationary.

In [ ]:
#here we take diff() to make gold data stationary.
diff_gold_data = gold_data.diff().dropna()
diff_gold_data.plot()

### AR (Auto Regression Model) :

In a auto regression model today's value is equal to mean plus fraction of yesterday's value plus noise.

Mathmetical description of AR(1) model :
&emsp; &emsp; &emsp; R<sub>t</sub> = $\mu$ &emsp; +  &emsp;$\phi$R<sub>t-1</sub> &emsp;+ &emsp;$\epsilon$<sub>t</sub>
<br>
AR parameter $\phi$ :
- if $\phi$ = 0 then R<sub>t</sub> is simply a white noise.
- if $\phi$ = 1 then R<sub>t</sub> is a random walk.
- for stationary process -1 < $\phi$ < 1

if $\phi$ < 0 then Mean reversion
and if $\phi$ = 1 then Momentum which we discuss previously.
<br><br>

<b> Higher order AR models : </b><br>
&emsp; &emsp; &emsp;AR(1) : &emsp; &emsp; &emsp; R<sub>t</sub> = $\mu$ &emsp; +  &emsp;$\phi$1R<sub>t-1</sub> &emsp;+ &emsp;$\epsilon$<sub>t</sub>
<br>
&emsp; &emsp; &emsp;AR(2) : &emsp; &emsp; &emsp; R<sub>t</sub> = $\mu$ &emsp; +  &emsp;$\phi$1R<sub>t-1</sub> &emsp;+&emsp;$\phi2$R<sub>t-2</sub> &emsp;+ &emsp;$\epsilon$<sub>t</sub>
<br>
&emsp; &emsp; &emsp;AR(3) : &emsp; &emsp; &emsp; R<sub>t</sub> = $\mu$ &emsp; +  &emsp;$\phi$1R<sub>t-1</sub> &emsp;+ &emsp;$\phi$2R<sub>t-2</sub> &emsp;+&emsp;$\phi$3R<sub>t-3</sub> &emsp;+&emsp;$\epsilon$<sub>t</sub>
<br>
<b>...</b>

### Estimating a AR Model :

In [ ]:
from statsmodels.tsa.arima_model import ARMA
model = ARMA(gold_data,order=(1,0)) #here order attribute is used to select AR model
result = model.fit()
print(result.summary())

### Choosing a Right Model :
Identifying the right order AR mdoel gives us best AR model to forcast the values.
- The order of an AR model ususally be unkown.
- Two techniques to determine order : 
        - Partial autocorrelation function (PACF)
        - Information Criteria

### Partial Autocorrelation Function :
Partial autocorelation function measures the incremental benefit of adding lag. Basically it provides us information if we add one more lag to our model then it would benefit the model or not.

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(gold_data['gold_price'],lags=20,alpha=0.10)
plt.show()

In this graph we can see that after lag 1 partical autocorrelation is approx zero. So we can use only one lag in the model.

### Information Criteria :
Generally if we have higher order AR model then it can be better fit for our dataset. But may result model overfit our dataset.<br>
So information is used to by adjusting goodness of fit measures by imposing penalty based on number of parameters that are used.Two popular adjusted goodness of fit measures.
- AIC (Akaike Information Criteria)
- BIC (Baysian Information Criteria)

We choose the model which have lowest aic or bic values.

In [ ]:
parameters = [i for i in range(1,6)]
bic_values = []
for p in parameters:
    model = ARMA(gold_data,order=(p,0))
    result = model.fit()
    bic_values.append(result.bic)
plt.plot(bic_values)
plt.xticks(parameters)
plt.xlabel('No. of Parameter')
plt.ylabel('BIC value')
plt.show()

In this graph for order 1 bic value is minimum then order one AR model is best for our series to forcast.

### Forcasting future value with AR model

In [ ]:
model = ARMA(gold_data,order=(1,0))
result = model.fit()
'''Start the forecast 10 data points before the end of the 2290 point series at 2280,and 
end the forecast 10 data points after the end of the series at point 2300'''
result.plot_predict(start=2280, end=2300)
plt.xticks()
plt.plot()



### MA (Moving Average) Model : 

In a auto regression model today's value is equal to mean plus noise plus fraction of yesterday's noise.

Mathmetical description of MR(1) model :
&emsp; &emsp; &emsp; R<sub>t</sub> = $\mu$ &emsp; + &emsp;$\epsilon$<sub>t</sub>&emsp;+  &emsp;$\theta$$\epsilon$<sub>t-1</sub> &emsp;
<br>

Interpretation of MR parameter $\theta$ :
- if $\theta$ = 0 then R<sub>t</sub> is simply a white noise.
- MA model is stationary for all values of $\theta$


if $\theta$ is negative : One period time Reversion
and if $\theta$ is positive :  One period time Momentum
<br><br>

<b>Note :</b> One period autocorrelation is $\theta$/(1+$\theta^2$). Not $\theta$.

<b> Higher order MA models : </b><br>
&emsp; &emsp; &emsp;MA(1) : &emsp; &emsp; &emsp; R<sub>t</sub> = $\mu$ &emsp; + &emsp;$\epsilon$<sub>t</sub> &emsp;- &emsp;$\theta$<sub>1</sub>$\epsilon$<sub>t-1</sub> &emsp;
<br>
&emsp; &emsp; &emsp;MA(2) : &emsp; &emsp; &emsp; R<sub>t</sub> = $\mu$ &emsp; + &emsp;$\epsilon$9 &emsp;-  &emsp;$\theta$<sub>1</sub>$\epsilon$<sub>t-1</sub> &emsp;-  &emsp;$\theta$<sub>2</sub>$\epsilon$<sub>t-2</sub> &emsp;
<br>
&emsp; &emsp; &emsp;MA(3) : &emsp; &emsp; &emsp; R<sub>t</sub> = $\mu$ &emsp; + &emsp;$\epsilon$<sub>t</sub> &emsp;-  &emsp;$\theta$<sub>1</sub>$\epsilon$<sub>t-1</sub> &emsp;-  &emsp;$\theta$<sub>2</sub>$\epsilon$<sub>t-2</sub> &emsp;-  &emsp;$\theta$<sub>3</sub>$\epsilon$<sub>t-3</sub> &emsp;
<br>
<b>...</b><br>
<small>Many textbooks and software programs define the model with negative signs before the  terms. This doesn’t change the general theoretical properties of the model, although it does flip the algebraic signs of estimated coefficient values and (unsquared)  terms in formulas for ACFs and variances. You need to check your software to verify whether negative or positive signs have been used in order to correctly write the estimated model. R uses positive signs in its underlying model, as we do here.</small>

### Estimating a MA Model :

In [ ]:
#order 1 MA model
from statsmodels.tsa.arima_model import ARMA
model = ARMA(gold_data,order=(0,1)) #here order attribute is used to select MR model
result = model.fit()
print(result.summary())

Unlike an AR(1), an MA(1) model has no autocorrelation beyond lag 1, an MA(2) model has no autocorrelation beyond lag 2, etc. The lag-1 autocorrelation for an MA(1) model is not θ, but rather θ/(1+$θ^2$). For example, if the MA parameter, θ, is = +0.9, the first-lag autocorrelation will be 0.9/(1+$(0.9)^2$)=0.497, and the autocorrelation at all other lags will be zero. If the MA parameter, θ, is -0.9, the first-lag autocorrelation will be −0.9/(1+$(−0.9)^2$)=−0.497.

In [ ]:
#partial autocorrelation function plot
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(gold_data, lags=20)
plt.show()

In the above plot we can see only lag 1 is showing partial autocorrelation.

### Forcasting with a MA model :

In [ ]:
model = ARMA(gold_data,order=(0,1))
result = model.fit()
'''Start the forecast 10 data points before the end of the 2290 point series at 2280,and 
end the forecast 10 data points after the end of the series at point 2300'''
result.plot_predict(start=2280, end=2300)
plt.xticks()
plt.plot()


### ARMA Models: 
An ARMA model is a combination of AR model and MA model.

- Equation of order 1 ARMA model or ARMA(1,1) model : <br>
 &emsp; &emsp; &emsp; &emsp;&emsp;R<sub>t</sub> &emsp; = &emsp; $\mu$ &emsp; +  &emsp;$\phi$R<sub>t-1</sub> &emsp;+ &emsp;$\epsilon$<sub>t</sub> &emsp;+&emsp;$\theta$R<sub>t-1</sub> &emsp;
<br>

### Fitting and Forcasting for an ARMA Model :

In [ ]:
model = ARMA(gold_data,order=(1,1)) # order =(1,1) is used to select order 1 ARMA model
result = model.fit()
'''Start the forecast 10 data points before the end of the 2290 point series at 2280,and 
end the forecast 10 data points after the end of the series at point 2300'''
result.plot_predict(start=2280, end=2300)
plt.xticks()
plt.plot()

We can select best order for our model by using partial auto correlation funtion and information criteria which I peformed for AR model.

I hope you will get familiar with time series and some basic concepts of time series. Now you can apply different time series models on different time series data and forcast furture value for them. Time series is useful for  Economic Forecasting,
Sales Forecasting,
Budgetary Analysis,
Stock Market Analysis,
Yield Projections,
Process and Quality Control,
Inventory Studies,
Workload Projections and many other ares also.

**> *If you like this note-book upvote this so that I konw you are able to gain some knowledge through this note and there is some mistake or improvment you found, then you can comment your valuable thoughts.***